In [2]:
pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 129.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 6.3 MB/s eta 0:00:00


Aquí te explico brevemente cómo sería una política de IAM que otorgue este permiso. Deberás adjuntar esta política a tu usuario de AWS en la consola de IAM:

In [11]:
# Use the native inference API to send a text message to Mistral.

import boto3
import json
from botocore.exceptions import ClientError
from google.colab import userdata # Import userdata to access Colab secrets

# Create a Bedrock Runtime client in the AWS Region of your choice.
# Ensure your AWS credentials are configured in your environment (e.g., via environment variables, shared credential file, or IAM roles).

# Get AWS credentials from Colab secrets
aws_access_key_id = userdata.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = userdata.get('AWS_SECRET_ACCESS_KEY')
# Optionally, you can also get the region from secrets
# aws_region = userdata.get('AWS_REGION', 'us-east-1') # Default to us-east-1 if not in secrets


# Create the boto3 client using the credentials from secrets
# The Bedrock API key concept is handled through AWS IAM permissions and credentials
# We use the AWS access key ID and secret access key to authenticate
client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1", # Replace with aws_region if you get it from secrets
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)


# Set the model ID, e.g., Mistral Large.
model_id = "mistral.mistral-7b-instruct-v0:2"

# Define the prompt for the model.
prompt = "Describe the purpose of a 'hello world' program in one line."

# Embed the prompt in Mistral's instruction format.
formatted_prompt = f"<s>[INST] {prompt} [/INST]"

# Format the request payload using the model's native structure.
native_request = {
    "prompt": formatted_prompt,
    "max_tokens": 512,
    "temperature": 0.5,
}

# Convert the native request to JSON.
request = json.dumps(native_request)

try:
    # Invoke the model with the request.
    response = client.invoke_model(modelId=model_id, body=request)

    # Decode the response body.
    model_response = json.loads(response["body"].read())

    # Extract and print the response text.
    response_text = model_response["outputs"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    # The NameError is avoided by printing the error message here instead of trying to access 'response'
    # exit(1) # Removed exit(1) to allow the notebook to continue running

 A "Hello World" program is an introductory tutorial example in programming to demonstrate the basic syntax, structure and output of a programming language.


In [1]:
! pip install faiss-cpu==1.7.4 mistralai "numpy<2"

In [3]:
from mistralai import Mistral
import requests
import numpy as np
import faiss
import os
from getpass import getpass
from google.colab import userdata

api_key= userdata.get('MISTRAL_SECRET_API_KEY')
client = Mistral(api_key=api_key)

In [4]:
response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

In [5]:
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

In [6]:
def get_text_embedding(input):
    embeddings_batch_response = client.embeddings.create(
          model="mistral-embed",
          inputs=input
      )
    return embeddings_batch_response.data[0].embedding

In [7]:
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

In [8]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [9]:
question = "What were the two main things the author worked on before college?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape

(1, 1024)

In [10]:
D, I = index.search(question_embeddings, k=2)
print(I)

[[0 3]]


In [11]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['\n\nWhat I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then 

In [12]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [13]:
def run_mistral(user_message, model="mistral-large-latest"):
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

In [17]:
run_mistral(prompt)

'Before college, the author worked on writing and programming.'

In [20]:
!pip install -qU langchain langchain-openai langchain-tavily

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 30.4 MB/s eta 0:00:00


In [23]:
# !pip install -qU langchain langchain-openai langchain-tavily
from typing import Any, Dict, Optional
import datetime
import os

from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langchain.schema import HumanMessage, SystemMessage
from google.colab import userdata # Import userdata to access Colab secrets

# Initialize LLM
# Get OpenAI API key from Colab secrets
openai_api_key = userdata.get('OPENAI_API_KEY')
llm = init_chat_model(model="gpt-4o", model_provider="openai", temperature=0, openai_api_key=openai_api_key)

# Initialize Tavily Search Tool
# Get Tavily API key from Colab secrets
tavily_api_key = userdata.get('TAVILY_SECRET_API_KEY')
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
    tavily_api_key=tavily_api_key
)

# Set up Prompt with 'agent_scratchpad'
today = datetime.datetime.today().strftime("%D")
prompt = ChatPromptTemplate.from_messages([
    ("system", f"""You are a helpful reaserch assistant, you will be given a query and you will need to
    search the web for the most relevant information. The date today is {today}."""),
    MessagesPlaceholder(variable_name="messages"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),  # Required for tool calls
])

# Create an agent that can use tools
agent = create_openai_tools_agent(
    llm=llm,
    tools=[tavily_search_tool],
    prompt=prompt
)

# Create an Agent Executor to handle tool execution
agent_executor = AgentExecutor(agent=agent, tools=[tavily_search_tool], verbose=True)

user_input =  "What is the most popular sport in the world? include only wikipedia sources"

# Construct input properly as a dictionary
response = agent_executor.invoke({"messages": [HumanMessage(content=user_input)]})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': 'most popular sport in the world', 'include_domains': ['wikipedia.org']}`


{'query': 'most popular sport in the world', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://en.wikipedia.org/wiki/Sport', 'title': 'Sport - Wikipedia', 'content': "The world's most accessible and practised sport is running, while association football is the most popular spectator sport.", 'score': 0.7539928, 'raw_content': None}, {'url': 'https://en.wikipedia.org/wiki/Association_football', 'title': 'Association football - Wikipedia', 'content': "With an estimated 250 million players active in over 200 countries and territories, it is the world's most popular sport. Association football. The attacking", 'score': 0.68874115, 'raw_content': None}, {'url': 'https://en.wikipedia.org/wiki/Sports_in_the_United_States', 'title': 'Sports in the United States - Wikipedia', 'content': 'Based on reve